In [2]:
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
import pandas as pd

Using TensorFlow backend.


In [3]:
import matplotlib
import matplotlib.pyplot as plt
import cv2
import scipy
matplotlib.rcParams['figure.figsize'] = [15, 15]

In [4]:
import keras
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input, Softmax
from keras.layers import Conv2D, MaxPooling2D, SeparableConv2D
from keras.layers import LeakyReLU

In [5]:
target_shape = (17, 22)

In [6]:
input_shape = (17, 22, 3)
alpha = 0.2

inputs = Input(shape=input_shape)

x = Conv2D(8, (3, 3))(inputs)
x = LeakyReLU(alpha)(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(16, (3, 3))(inputs)
x = LeakyReLU(alpha)(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(32, (3, 3))(x)
x = LeakyReLU(alpha)(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)

x = Dense(64)(x)
x = LeakyReLU(alpha)(x)

x = Dense(32)(x)
x = LeakyReLU(alpha)(x)

x = Dense(10, activation="softmax")(x)
outputs = x

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"])

In [7]:
image_gen = ImageDataGenerator(validation_split=0.15, rescale=1.0/255.0, width_shift_range=4, height_shift_range=2, zoom_range=0.1)
dataframe = pd.read_csv("../assets/data/values/labels.txt", delim_whitespace=True)
digit_filter = (dataframe['category'] >= 0) & (dataframe['category'] <= 9)
dataframe = dataframe[digit_filter]
dataframe = dataframe.astype(str)

In [8]:
training_generator = image_gen.flow_from_dataframe(
    dataframe=dataframe, 
    directory="../assets/data/values/",
    x_col="filename",
    y_col="category",
    subset="training",
    batch_size=20,
    classes=[str(i) for i in range(0,10)],
    class_mode="categorical",
    shuffle=True,
    target_size=target_shape
)

Found 5193 validated image filenames belonging to 10 classes.


In [9]:
validation_generator = image_gen.flow_from_dataframe(
    dataframe=dataframe, 
    directory="../assets/data/values/",
    x_col="filename",
    y_col="category",
    subset="validation",
    batch_size=20,
    classes=[str(i) for i in range(0,10)],    
    class_mode="categorical",
    shuffle=True,
    target_size=target_shape
)

Found 916 validated image filenames belonging to 10 classes.


In [65]:
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 17, 22, 3)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 15, 20, 16)        448       
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 15, 20, 16)        0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 7, 10, 16)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 5, 8, 32)          4640      
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 5, 8, 32)          0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 2, 4, 32)          0   

In [67]:
steps_per_epoch = training_generator.n//training_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

model.fit_generator(
    generator=training_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=15,
)

Epoch 1/15
253/253 [==============================] - 6s 24ms/step - loss: 0.2838 - accuracy: 0.9190 - val_loss: 0.4014 - val_accuracy: 0.9170
Epoch 2/15
253/253 [==============================] - 7s 29ms/step - loss: 0.2640 - accuracy: 0.9226 - val_loss: 0.1134 - val_accuracy: 0.9199
Epoch 3/15
253/253 [==============================] - 7s 27ms/step - loss: 0.2444 - accuracy: 0.9299 - val_loss: 0.2336 - val_accuracy: 0.9199
Epoch 4/15
253/253 [==============================] - 7s 27ms/step - loss: 0.2430 - accuracy: 0.9307 - val_loss: 0.2950 - val_accuracy: 0.9199
Epoch 5/15
253/253 [==============================] - 9s 36ms/step - loss: 0.2327 - accuracy: 0.9337 - val_loss: 0.2658 - val_accuracy: 0.9073
Epoch 6/15
253/253 [==============================] - 6s 25ms/step - loss: 0.2291 - accuracy: 0.9347 - val_loss: 0.1831 - val_accuracy: 0.9245
Epoch 7/15
253/253 [==============================] - 7s 29ms/step - loss: 0.2170 - accuracy: 0.9356 - val_loss: 0.2171 - val_accuracy: 0.9222

In [1]:
#model.save("values.h5")
#model.save("../assets/models/values.h5")